EVA优化器+SGD优化器，在batch_size = 32时优化效果展示：

In [2]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
from kfac import EVA
from fused import FusedAdam
#from fused import Fused_KFAC
from fused.fused_eva.fused_eva import Fused_KFAC
transform = transforms.Compose(
    
    
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net.to(device)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = EVA(net, lr=0.001)
preconditioner = EVA(net, lr=0.001)
#optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
#optimizer = FusedAdam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

#preconditioner = Fused_KFAC(net, lr=0.001)
import time
since = time.time()

# # # define timer
forward_start = torch.cuda.Event(enable_timing=True)
forward_end = torch.cuda.Event(enable_timing=True)

backward_start = torch.cuda.Event(enable_timing=True)
backward_end = torch.cuda.Event(enable_timing=True)

optimize_start = torch.cuda.Event(enable_timing=True)
optimize_end = torch.cuda.Event(enable_timing=True)

sum_forward_time = 0
sum_backward_time = 0
sum_optimize_time = 0

num = 0
print('success fused')
# print(net.parameters())
# for p in net.parameters():
# #     print(p)
#     print(p.numel())
    
# print(len(optimizer.param_groups[0]['params']))
# print(optimizer.param_groups[0]['params'])

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        forward_start.record()
        outputs = net(inputs)
        torch.cuda.synchronize()
        forward_end.record()
        
        loss = criterion(outputs, labels)
        
        backward_start.record()
        loss.backward()
        torch.cuda.synchronize()
        backward_end.record()
        
        optimize_start.record()
        preconditioner.step()
        optimizer.step()
        torch.cuda.synchronize()
        optimize_end.record()
        # print statistics
        
        running_loss += loss.item()
        if num > 20:
            sum_forward_time += forward_start.elapsed_time(forward_end)
            sum_backward_time += backward_start.elapsed_time(backward_end)
            sum_optimize_time += optimize_start.elapsed_time(optimize_end)
        num += 1
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            time_elapsed = time.time() - since
            print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
   # print('when set foreach = False, fused = False, calculate the average time an iteration takes:')
num = num -20
print('when set EVA optimizer, calculate the average time an iteration takes:')
print('batch_size = {:.0f}'.format(batch_size))
print('foward complete in {:.10f}s'.format(sum_forward_time/num))
print('backward complete in {:.10f}s'.format(sum_backward_time/num))
print('optimize complete in {:.10f}s'.format(sum_optimize_time/num))
print('Finished Training')
nogrouped = sum_optimize_time/num

Files already downloaded and verified
Files already downloaded and verified
cuda:0
success fused
when set EVA optimizer, calculate the average time an iteration takes:
batch_size = 32
foward complete in 1.1358306346s
backward complete in 1.0092008751s
optimize complete in 2.0404297775s
Finished Training


FusedEVA优化器+SGD优化器，在batch_size = 32时优化效果展示：

In [3]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
from kfac import EVA
from fused import FusedAdam
#from fused import Fused_KFAC
from fused import FusedEVA
from fused.fused_eva.fused_eva import Fused_KFAC
import os

# 设置 CUDA_LAUNCH_BLOCKING 环境变量为 1
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
transform = transforms.Compose(

    
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
net.to(device)
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = EVA(net, lr=0.001)
#preconditioner = EVA(net, lr=0.001)
#optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
#optimizer = FusedAdam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

preconditioner = FusedEVA(net, lr=0.001)
import time
since = time.time()

# # # define timer
forward_start = torch.cuda.Event(enable_timing=True)
forward_end = torch.cuda.Event(enable_timing=True)

backward_start = torch.cuda.Event(enable_timing=True)
backward_end = torch.cuda.Event(enable_timing=True)

optimize_start = torch.cuda.Event(enable_timing=True)
optimize_end = torch.cuda.Event(enable_timing=True)

sum_forward_time = 0
sum_backward_time = 0
sum_optimize_time = 0

num = 0
print('success fused')
# print(net.parameters())
# for p in net.parameters():
# #     print(p)
#     print(p.numel())
    
# print(len(optimizer.param_groups[0]['params']))
# print(optimizer.param_groups[0]['params'])

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        forward_start.record()
        outputs = net(inputs)
        torch.cuda.synchronize()
        forward_end.record()
        loss = criterion(outputs, labels)
        
        backward_start.record()
        loss.backward()
        torch.cuda.synchronize()
        backward_end.record()
        
        optimize_start.record()
        preconditioner.step()
        optimizer.step()
        torch.cuda.synchronize()
        optimize_end.record()
        # print statistics
        
        running_loss += loss.item()
        if num > 20:
            sum_forward_time += forward_start.elapsed_time(forward_end)
            sum_backward_time += backward_start.elapsed_time(backward_end)
            sum_optimize_time += optimize_start.elapsed_time(optimize_end)
        num += 1
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            time_elapsed = time.time() - since
            print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
   # print('when set foreach = False, fused = False, calculate the average time an iteration takes:')
num = num - 20
print('when set FusedEVA optimizer, calculate the average time an iteration takes:')
print('batch_size = {:.0f}'.format(batch_size))
print('foward complete in {:.10f}s'.format(sum_forward_time/num))
print('backward complete in {:.10f}s'.format(sum_backward_time/num))
print('optimize complete in {:.10f}s'.format(sum_optimize_time/num))
print('Finished Training')
grouped = sum_optimize_time/num

Files already downloaded and verified
Files already downloaded and verified
cuda:0
success fused
when set FusedEVA optimizer, calculate the average time an iteration takes:
batch_size = 32
foward complete in 1.1596944228s
backward complete in 0.9988078648s
optimize complete in 1.7696022948s
Finished Training


In [5]:
print('speed up {:.10f}'.format(nogrouped/grouped))

speed up 1.1530442651
